In [1]:
import pandas as pd
import numpy as np
import cx_Oracle
import os
import json
import datetime
import warnings

warnings.filterwarnings(action="ignore")
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import bs4
import requests
import time
import math
import random
from shapely.geometry import Point, Polygon
import time as timesys
from pyproj import Geod
from haversine import haversine
pd.set_option('display.max_columns', None)

In [2]:
tianjin = '117.4961975657171,37.86803012379458,7.296245301137998 119.8081837995413,37.9250916349774,-34.2681049860788 119.8416643486643,39.32481316399956,-31.12524274095152 117.9501096884913,39.37523731857854,39.41212136434525 117.4961975657171,37.86803012379458,7.296245301137998'



coord_list = tianjin.split(" ")
coord_list
coord_list2 = []
for i in coord_list:
    temp = i.split(",")[:2]
    temp = list(map(float, temp))
    coord_list2.append(temp)
poly = Polygon(coord_list2)
# if Point(LONGITUDE, LATITUDE).within(poly):

정상 항차   
26,25,24,23,20

In [3]:
df_test = pd.read_csv("voy_path/COSCO_FOS_VRKR6_20.csv")
df_test

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption
0,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-09-29 01:00:00,125.9787,34.1164,20.0,265.0,2022-09-28 02:34:41.172206,2022-09-27 17:33:59,2022-09-27 01:31:10,under way using engine,276.931761,613.081865
1,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-09-29 01:00:00,125.7783,34.1131,20.0,291.0,2022-09-28 03:04:43.418897,2022-09-27 18:04:29,2022-09-27 01:31:10,under way using engine,294.364669,651.675487
2,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-09-29 01:00:00,125.6743,34.2355,19.8,352.0,2022-09-28 03:34:46.226145,2022-09-27 18:34:22,2022-09-27 01:31:10,under way using engine,299.196674,652.772691
3,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-09-29 01:00:00,125.6468,34.3992,19.4,352.0,2022-09-28 04:04:49.296380,2022-09-27 19:04:34,2022-09-27 01:31:10,under way using engine,296.931314,629.172829
4,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-09-29 01:00:00,125.6274,34.5606,19.4,355.0,2022-09-28 04:34:52.633460,2022-09-27 19:34:41,2022-09-27 07:25:07,under way using engine,295.094039,625.279795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-10-04 10:00:00,128.8250,35.0774,0.1,91.0,2022-10-04 23:09:58.929655,2022-10-04 14:09:15,2022-10-04 02:07:09,moored,11.997479,957.304365
243,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-10-04 10:00:00,128.8255,35.0773,0.0,91.0,2022-10-04 23:40:01.478256,2022-10-04 14:36:10,2022-10-04 02:31:07,moored,11.988375,NaN
244,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-10-04 10:00:00,128.8252,35.0776,0.1,91.0,2022-10-05 02:40:17.931761,2022-10-04 17:36:10,2022-10-04 05:37:08,moored,12.020477,959.139487
245,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-10-04 10:00:00,128.8251,35.0773,0.1,91.0,2022-10-05 03:40:24.322710,2022-10-04 18:39:15,2022-10-04 06:25:09,moored,11.986756,956.448815


In [4]:
# if Point(LONGITUDE, LATITUDE).within(poly):

In [5]:
df_test.sort_values(by="timestamp", ignore_index=True, inplace=True)

In [6]:
for idx in df_test.index:
    LONGITUDE = df_test.loc[idx, "longitude"]
    LATITUDE = df_test.loc[idx, "latitude"]
    heading = df_test.loc[idx, "heading"]
    speed = df_test.loc[idx, "speed"]
    if Point(LONGITUDE, LATITUDE).within(poly) and heading < 200 and speed > 8:
#         print(idx)
        break
        

In [7]:
df_test_2 = df_test.loc[idx:,]
df_test_2.reset_index(inplace=True)

In [8]:
# df_test_2.to_csv("voy_path/COSCO_test.csv", encoding="utf-8 sig", header=True, index=False)

In [9]:
distance = 0
for idx in df_test_2.index:
    if idx != 0:
        LONGITUDE = df_test_2.loc[idx, "longitude"]
        LATITUDE = df_test_2.loc[idx, "latitude"]
        current_coord = (LATITUDE, LONGITUDE)

        LONGITUDE = df_test_2.loc[idx - 1, "longitude"]
        LATITUDE = df_test_2.loc[idx - 1, "latitude"]
        previous_current_coord = (LATITUDE, LONGITUDE)

        distance += haversine(current_coord, previous_current_coord, unit="km")

In [10]:
distance

1390.2571841055944

In [11]:
1348.5265119662952 / 20

67.42632559831476

In [12]:
overall_length = 1348.5265119662952
segment_length = overall_length/20

In [13]:
overall_length = distance
segment_length = overall_length/20

In [14]:
segment_length

69.51285920527972

In [15]:
temp_accu_length = 0
segmenting_index_list = []
for idx in df_test_2.index:
    
    if idx == 0:
        previous_length = 0
    else:
        LONGITUDE = df_test_2.loc[idx, "longitude"]
        LATITUDE = df_test_2.loc[idx, "latitude"]
        current_coord = (LATITUDE, LONGITUDE)

        LONGITUDE = df_test_2.loc[idx - 1, "longitude"]
        LATITUDE = df_test_2.loc[idx - 1, "latitude"]
        previous_current_coord = (LATITUDE, LONGITUDE)

        current_length = haversine(current_coord, previous_current_coord, unit="km")
        temp_accu_length += current_length
        
        if temp_accu_length >= segment_length:
            print(temp_accu_length)
#             segmenting_index_list.append(idx)
            
            
            length_error_1 = temp_accu_length - segment_length
            length_error_2 = temp_accu_length - segment_length - previous_length
            if length_error_1 > length_error_2:
                segmenting_index_list.append(idx)
            else:
                segmenting_index_list.append(idx - 1)
                
            
            temp_accu_length = 0
            
        previous_length = current_length
segmenting_index_list.append(df_test_2.index[-1])


75.3054712478748
72.94456453129513
79.66698682908542
71.69434394911764
82.71890231447972
72.48446084634031
77.05866363402825
69.69870021954273
70.60262220298297
80.34519855421348
70.83588533026976
74.91218070196437
74.00463924806776
74.4313408905518
81.82909180878984
80.47227001213217
81.38490143384719
72.73359381958534


In [16]:
len(segmenting_index_list)

19

In [17]:
# df 내의 포인트 전체 길이 구하는 함수
def reckon_point_lengths(df_test_2):
    distance = 0
    df_test_2.reset_index(inplace=True, drop=True)
    for idx in df_test_2.index:
        if idx != 0:
            LONGITUDE = df_test_2.loc[idx, "longitude"]
            LATITUDE = df_test_2.loc[idx, "latitude"]
            current_coord = (LATITUDE, LONGITUDE)

            LONGITUDE = df_test_2.loc[idx - 1, "longitude"]
            LATITUDE = df_test_2.loc[idx - 1, "latitude"]
            previous_current_coord = (LATITUDE, LONGITUDE)

            distance += haversine(current_coord, previous_current_coord, unit="km")
    return distance

In [18]:
for idx, index in enumerate(segmenting_index_list):
    if idx == 0:
        d = df_test_2[0:index + 1]
#         display(d)
        distance = reckon_point_lengths(d)
        print(distance)
        d.to_csv(f"실험/segment_test_new_20_{idx}.csv", encoding="utf-8 sig", header=True, index=False)
    else:
        d = df_test_2[previous_index:index + 1]
#         display(d)
        distance = reckon_point_lengths(d)
        print(distance)
        d.to_csv(f"실험/segment_test_new_20_{idx}.csv", encoding="utf-8 sig", header=True, index=False)
        
    previous_index = index

75.3054712478748
72.94456453129513
79.66698682908542
71.69434394911764
82.71890231447972
72.48446084634031
77.05866363402825
69.69870021954273
70.60262220298297
80.34519855421348
70.83588533026976
74.91218070196437
74.00463924806776
74.4313408905518
81.82909180878984
80.47227001213217
81.38490143384719
72.73359381958534
27.13336653142446


In [19]:
for i in range(len(segmenting_index_list)):
    d = pd.read_csv(f"실험/segment_test_new_20_{i}.csv")
    mean_speed = d["speed"].mean()
    print(i, mean_speed)

0 8.2
1 9.370000000000001
2 13.614285714285714
3 12.457142857142857
4 12.8125
5 13.099999999999998
6 11.825
7 12.557142857142859
8 12.657142857142858
9 12.462499999999999
10 3.6913043478260876
11 2.79375
12 11.4375
13 11.649999999999999
14 12.557142857142855
15 12.625
16 12.525
17 4.547619047619047
18 3.5499999999999994


In [20]:
print("COSCO_FOS_20")
print("전항지 출항 시각",df_test_2.loc[df_test_2.index[0], "timestamp"])
print("부산 도착 시각",df_test_2.loc[df_test_2.index[-1], "timestamp"])

COSCO_FOS_20
전항지 출항 시각 2022-09-30 23:00:41.873653
부산 도착 시각 2022-10-05 04:40:30.462938
